In [1]:
import keras
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

import multiprocessing
import numpy as np
import pickle
import time

Using TensorFlow backend.


In [2]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [3]:
# Data scaling
out_min = -1
out_max = 1

X_scaled = np.copy(X)

x_min = np.min(X)
x_max = np.max(X)

cnt = 0
for xi in X:
    x_std = (xi - x_min)/(x_max - x_min)
    x_scld = x_std*(out_max - out_min) + out_min
    X_scaled[cnt] = x_scld
    cnt = cnt + 1

print(X_scaled.shape)
print(np.min(X_scaled))
print(np.max(X_scaled))

(414, 100, 100, 2)
-1.0
1.0


In [4]:
def run_training(X_scaled, y, NAME, layer_size, conv_layer, dense_layer):
    model = keras.Sequential()
    
    model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    for l in range(conv_layer-1):
        model.add(Conv2D(layer_size, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
    model.add(Flatten())
    
    for _ in range(dense_layer):
        model.add(Dense(layer_size))
        model.add(Activation('relu'))
    
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    model.fit(X_scaled, y,
              batch_size=20,
              epochs=10,
              validation_split=0.2,
              callbacks=[tensorboard])
    
    model.save('32x2conv-0dense.model')
    model.summary()


layer_sizes = [32]
conv_layers = [2]
dense_layers = [0]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)
            
            p = multiprocessing.Process(target=run_training, args=(X_scaled, y, NAME, layer_size, conv_layer,
                                                                  dense_layer))
            p.start()
            p.join()

2-conv-32-nodes-0-dense-1594058260
Train on 331 samples, validate on 83 samples
Epoch 1/10
331/331 [==============================] - 1s 4ms/step - loss: 0.6274 - acc: 0.6647 - val_loss: 0.5194 - val_acc: 0.8072
Epoch 2/10
331/331 [==============================] - 0s 654us/step - loss: 0.4088 - acc: 0.8338 - val_loss: 0.2763 - val_acc: 0.9036
Epoch 3/10
331/331 [==============================] - 0s 657us/step - loss: 0.1967 - acc: 0.9335 - val_loss: 0.0972 - val_acc: 0.9880
Epoch 4/10
331/331 [==============================] - 0s 645us/step - loss: 0.0904 - acc: 0.9668 - val_loss: 0.0542 - val_acc: 1.0000
Epoch 5/10
331/331 [==============================] - 0s 646us/step - loss: 0.0601 - acc: 0.9789 - val_loss: 0.0175 - val_acc: 1.0000
Epoch 6/10
331/331 [==============================] - 0s 679us/step - loss: 0.0401 - acc: 0.9940 - val_loss: 0.0126 - val_acc: 1.0000
Epoch 7/10
331/331 [==============================] - 0s 667us/step - loss: 0.0202 - acc: 0.9970 - val_loss: 0.0180 - 